# Joint Networks:

In [ ]:
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm
import random

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from numpy import array
from keras.models import load_model

from keras.layers import Layer
import keras.backend as K

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import platform
import time
import pathlib
import os
from keras.callbacks import ModelCheckpoint

from pandas import read_csv
import numpy as np
from keras import Model
from keras.layers import Layer
import keras.backend as K
from keras.layers import Input, Dense, SimpleRNN
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.metrics import mean_squared_error

# Data Preparation:

In [2]:
#read all the data:
items = pd.read_csv("/content/item_features.csv")
purchase =  pd.read_csv("/content/train_purchases.csv")
sessions =  pd.read_csv("/content/train_sessions.csv")

In [3]:
import time
purchase['date'] = pd.to_datetime(purchase['date'])
sessions['date'] = pd.to_datetime(sessions['date'])
max_day = purchase['date'].max() #max date 31/05/2021 
min_day = max_day -datetime.timedelta(31)
cond = purchase['date'] >= min_day
new_purchases = purchase[cond]
top_seller = new_purchases['item_id'].value_counts()[0:100].index.to_list()

In [4]:
max_day = sessions['date'].max() #max date 31/05/2021 
testing_date = max_day -datetime.timedelta(31)
cond_test = sessions['date'] >= testing_date
cond_train = sessions['date'] < testing_date
cond_test_purchase = purchase.date >= testing_date
cond_train_purchase = purchase.date < testing_date
sessions_train = sessions[cond_train]
sessions_test = sessions[cond_test]
purchase_train = purchase[cond_train_purchase]
purchase_test = purchase[cond_test_purchase]

In [5]:
def sessions_list(sessions,purchase):
    antique_session = sessions.iat[0,0]
    list_sessions = []
    actual_session = []
    purchase_session = 0
    for i in tqdm(range(len(sessions))):
        new_session = sessions.iat[i,0]
        item = sessions.iat[i,1]
        if(new_session != antique_session):
            actual_session.append(purchase.iat[purchase_session,1])
            purchase_session += 1
            list_sessions.append(actual_session)
            actual_session = []
            antique_session = new_session
        actual_session.append(item)
    list_sessions.append(actual_session)
    return list_sessions

In [6]:
session_item = sessions_list(sessions,purchase)

100%|██████████| 4743820/4743820 [04:26<00:00, 17803.25it/s]


In [7]:
def training_sequences(session_item2):
    train_sequence = np.zeros((len(session_item2),6))
    trobat = False
    for i,x in tqdm(enumerate(session_item2)):
        length = len(x)
        llista = x.copy()
        if(length-1 < 5):
            for j in range(6-length):
                llista.insert(0,0)
        if(length-1 > 5):
            for j in range(length-6):
                llista.pop(0)
        llista.pop(-1)                                    
        train_sequence[i][0] = llista[0]
        train_sequence[i][1] = llista[1]
        train_sequence[i][2] = llista[2]
        train_sequence[i][3] = llista[3]
        train_sequence[i][4] = llista[4]
        train_sequence[i][5] = x[len(x)-1]
    return train_sequence

sequence_training = training_sequences(session_item)

1000000it [00:04, 228806.39it/s]


In [14]:
X = np.array(sequence_training)[:,:5] #sequence to predict from
y = np.array(sequence_training)[:,5:] #next item in the sequence
X = X.astype(int)
y = y.astype(int)
training = np.array(sequence_training)

# Network:

In [16]:
from keras.layers import Embedding, LSTM, Dense, Dropout, RNN, SimpleRNN, GRU, Attention, Conv1D, Conv2D
from keras.layers import Flatten, Conv2D

from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku

In [17]:
number_items = len(items.item_id.unique())
number_items += 1
number_items

23692

In [18]:
item_index = {}
index_item = {}
item_index[0] = 0
index_item[0] = 0
for i,x in enumerate(items.item_id.unique()):
    index_item[i+1] = x
    item_index[x] = i+1

In [19]:
def training_dataset(X2,y2):
    #first X.
    X_training = np.zeros((X2.shape[0],X2.shape[1]))
    y_training = np.zeros(y2.shape[0])
    
    for i in range(X2.shape[0]):
        for j in range(X2.shape[1]):
            X_training[i,j] = item_index[X2[i,j]]
    
    for i in range(y2.shape[0]):
        y_training[i] = item_index[y2[i][0]]
            
    return X_training, y_training
X_training, y_training = training_dataset(X,y)  
X_training = X_training.astype(int)
y_training = y_training.astype(int)

**Relevant Features:**

In [20]:
def get_item_features():
    items_features = {}
    new_item = 2
    features = []
    for i in tqdm(range(len(items))):
        if(items.iloc[i].item_id == new_item):
            features.append(items.iloc[i].feature_category_id)
            features.append(items.iloc[i].feature_value_id)
        else:
            items_features[new_item] = features
            new_item = items.iloc[i].item_id
            features = []
            features.append(items.iloc[i].feature_category_id)
            features.append(items.iloc[i].feature_value_id)
    items_features[new_item] = features
    return items_features
item_features = get_item_features()

100%|██████████| 471751/471751 [02:15<00:00, 3489.62it/s]


In [21]:
def get_item_features_simplified(item_features2,n_features = 66):
    items_features = np.zeros((items.item_id.nunique()+1,n_features))
    items_unique = items.item_id.unique()
    items_features[0] = np.zeros(n_features)
    for i,x in tqdm(enumerate(items_unique)):
        features = item_features2[x]
        items_features[i+1][0:len(features)] = features
    feature_training = np.zeros((X_training.shape[0],n_features,X_training.shape[1]))
    for i in tqdm(range(X_training.shape[0])):
        for j in range(X_training.shape[1]):
            feature_training[i,:,j] = items_features[X_training[i,j]]
            
    return feature_training
item_features_training = get_item_features_simplified(item_features)

23691it [00:00, 52015.24it/s]
100%|██████████| 1000000/1000000 [00:08<00:00, 116494.74it/s]


**LOSS**

In [22]:
#let's define a sparse categorical crossentropy to start!
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(
      y_true=labels,
      y_pred=logits,
      from_logits=True
    )

**Metrics**

In [23]:
#let's define a sparse categorical crossentropy to start!
from tensorflow.keras.utils import to_categorical
def top_5(y_true, y_pred):
    return  tf.keras.metrics.sparse_top_k_categorical_accuracy(
    y_true, y_pred, k=5
)
def top_50(y_true, y_pred):
    return  tf.keras.metrics.sparse_top_k_categorical_accuracy(
    y_true, y_pred, k=50
)
def top_100(y_true, y_pred):
    return  tf.keras.metrics.sparse_top_k_categorical_accuracy(
    y_true, y_pred, k=100
)

**TPU:**

In [24]:
tf.keras.backend.clear_session()

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

strategy = tf.distribute.experimental.TPUStrategy(resolver)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.36.226.194:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.36.226.194:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [25]:
total_item = items.item_id.nunique()

In [34]:
def double_NN(input_shape1, input_shape2, activation = "softmax", hidden_units = 128, num_items = total_item+1, item_features = 2):
    
    x1 = tf.keras.Input((input_shape1))
    
    y1 = tf.keras.Input((input_shape2,input_shape1))
    
    # Add Input Embedding Layer
    x2 = Embedding(num_items, 256, input_length=input_shape1)(x1)
    
    x3 = LSTM(512, return_sequences=False, 
          activation = "relu",  dropout=0.2,
    recurrent_dropout=0.1)(x2)

    x3 = LSTM(1024, return_sequences=False, 
          activation = "relu",  dropout=0.2,
    recurrent_dropout=0.1)(x2)

    y2 = Conv1D(filters=512, kernel_size= 2, strides = 2, activation='relu')(y1)
    
    y3 = Conv1D(filters=1024, kernel_size= 4, strides = 1, activation='relu')(y2)


    y4 = Flatten()(y3)
    

    x4 = Dense(512, trainable = True, activation = "relu")(x3)

    z = tf.concat([x3, y4], axis = 1)

    z = Flatten()(z)

    z2 = Dense(1024, trainable=True, activation="relu")(z)

    z3 = Dropout(0.3)(z2)

    outputs = Dense(num_items, trainable=True, activation="softmax")(z3)
    

    model= Model(inputs = [x1,y1], outputs = outputs)
    
    adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    model.compile(
          optimizer=adam_optimizer,
          loss=loss,
          metrics= ["acc",top_5,top_100],
          steps_per_execution=32)
    
    return model



In [35]:
model_d = double_NN(5,66)
model_d.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_12 (InputLayer)          [(None, 66, 5)]      0           []                               
                                                                                                  
 input_11 (InputLayer)          [(None, 5)]          0           []                               
                                                                                                  
 conv1d_10 (Conv1D)             (None, 33, 512)      5632        ['input_12[0][0]']               
                                                                                                  
 embedding_5 (Embedding)        (None, 5, 256)       6065152     ['input_11[0][0]']               
                                                                                            

In [36]:
with strategy.scope():
  model_d = double_NN(5,66)
  model_d.summary()
  model_d.fit(x = [X_training,item_features_training],y = y_training,epochs=150, batch_size=256, validation_split = 0.01, verbose=1,
          callbacks  = [EarlyStopping(monitor='val_acc', patience=10),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_acc', save_best_only=True)]
            )
model_d.save_weights('/tmp/best.h5', overwrite=True)

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_14 (InputLayer)          [(None, 66, 5)]      0           []                               
                                                                                                  
 input_13 (InputLayer)          [(None, 5)]          0           []                               
                                                                                                  
 conv1d_12 (Conv1D)             (None, 33, 512)      5632        ['input_14[0][0]']               
                                                                                                  
 embedding_6 (Embedding)        (None, 5, 256)       6065152     ['input_13[0][0]']               
                                                                                            

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


3868/3868 [==============================] - 101s 26ms/step - loss: 7.9782 - acc: 0.0261 - top_5: 0.0708 - top_100: 0.2753 - val_loss: 7.1747 - val_acc: 0.0499 - val_top_5: 0.1201 - val_top_100: 0.3877
Epoch 2/150
3868/3868 [==============================] - 67s 17ms/step - loss: 6.9893 - acc: 0.0525 - top_5: 0.1316 - top_100: 0.4162 - val_loss: 6.8344 - val_acc: 0.0669 - val_top_5: 0.1610 - val_top_100: 0.4577
Epoch 3/150
3868/3868 [==============================] - 66s 17ms/step - loss: 6.6054 - acc: 0.0670 - top_5: 0.1616 - top_100: 0.4752 - val_loss: 6.7344 - val_acc: 0.0773 - val_top_5: 0.1798 - val_top_100: 0.4833
Epoch 4/150
3868/3868 [==============================] - 67s 17ms/step - loss: 6.3340 - acc: 0.0768 - top_5: 0.1822 - top_100: 0.5132 - val_loss: 6.7235 - val_acc: 0.0829 - val_top_5: 0.1912 - val_top_100: 0.4938
Epoch 5/150
3868/3868 [==============================] - 67s 17ms/step - loss: 6.1159 - acc: 0.0851 - top_5: 0.1986 - top_100: 0.5461 - val_loss: 6.7853 - val_